In [ ]:
"""
The purpose of the following techniques is to create a program which takes in an image 
of four particles (3 plastic and 1 glass bead), 
and classifies the shape of the image based on the location of the glass bead. If it is on the short
axis the shape is 'c' for 'chevron', and if on the long axis it is 't' for 'top'.
"""

In [ ]:
"SOME TECHNIQUES USED"

In [ ]:
import matplotlib.pyplot as plt #for maknig plots inside the notebook
import skvideo.io # reads in videos
from skimage import measure, morphology #scikit modules containing the functions for measuring image properties
from skimage.filters import *  #image thresholding functions
#from skimage.util import invert
#from scipy import ndimage
from operator import attrgetter
import numpy as np
import math
from itertools import chain
from glob import glob

In [ ]:
"Reading a video"
# read in a video file, display the first and last image, return the video
def full_avi(filename):
    Vid = skvideo.io.vread(filename)

    fig, ax = plt.subplots(ncols=2)
    ax[0].imshow(Vid[0]) # first fame of video
    ax[1].imshow(Vid[-1]) #last frame of video

    print("Video shape is: ", Vid.shape)
    print("Video length is: ", len(Vid))
    
    return Vid

In [ ]:
# read video
filename = glob("./tchakamau/2*/test*.avi")[1]
print(filename)
frame_vid = full_avi(filename)

In [ ]:
# h is a list of indices for picking frames out of the video
h = list(range(0, len(frame_vid), int(len(frame_vid)/300)))
print('number of frames testing: %r' %(len(h)))

#take a sample of ~300 frames from the full video for testing 
framelist = frame_vid[h]
plt.imshow(framelist[-1]) #an example frame
print(framelist.shape)

In [ ]:
'Using thresholds to separate particles from bakground'
#How well the transparent particle is piked up depends on the threshold
#The following image shows some of the thresholds available
#I use the li and isodata thresholds in my image processing
img = framelist[1] #example image 
fig, ax = try_all_threshold(img, figsize=(10, 10), verbose=False)


In [ ]:
#Now you see every dot

In [ ]:
#now you see only the plastic ones

There is not suh a big differene between these,
but with a few functions to fill in the whole space we can get the whole raft from threshold_li.
The threshold_li has a better chance of capturing the entire outer edge, which allows us to do this
The others, like threshold_yen, sometimes capture refletons as well.

Measure.regionprops returns properties of a region, like its filled area, 
its orientation, its major axis, etc.
but it needs a labelled image, so we label the thresholded image first. 
Regionprops calculates a list of properties for every labelled area.

In [ ]:
thresh_img = img > threshold_li(img) # binary image
img_labelled = measure.label(thresh_img) # contains connected regions
properties_lists = measure.regionprops(img_labelled, coordinates = 'rc') 
plt.imshow(img_labelled)
print(properties_lists[0])

Measure.label picks out regions of the image whicha re connected and labels each conneted region
this way. To get the properties of the raft and not the dust on the screen or disattached partiles, we have to take the largest region only.

In [ ]:
biggest_region_properties = [max(list_of_properties, key = attrgetter('area')) for list_of_properties in properties_lists]
print(biggest_region_properties)

In [ ]:
A good way to separate out frames with rafts which have broken apart is by area. 
A good threshold (found on test data) is area >= 1875 pixels. 
Similarly, other parameters 